<a href="https://colab.research.google.com/github/takumi-maker/dmd/blob/main/dnn_limitorder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import talib
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import japanize_matplotlib

# 乱数シードを固定して結果の再現性を確保
np.random.seed(42)
torch.manual_seed(42)

# グラフのサイズを設定
plt.rcParams['figure.figsize'] = [12, 6]

# GmoFetcherを用いて取得したGMOコインの15分足のOHLCVデータを読み込む
ohlcv = pd.read_pickle('df_ohlcv_all.pkl')

# 特徴量（ATRとSTDDEV）を計算し、1つ前のデータにシフト
features = pd.DataFrame({
    'ATR': talib.ATR(ohlcv.hi, ohlcv.lo, ohlcv.cl),
    'STDDEV': talib.STDDEV(ohlcv.cl),
}).shift(1)

# ターゲットデータ（open, low, close）を準備
target_data = pd.DataFrame({
    'open': ohlcv.op,
    'low': ohlcv.lo,
    'close': ohlcv.cl
})

# 欠損値を除去してデータを整える
features = features.dropna()
target_data = target_data.loc[features.index]

# データを学習用とテスト用に分割
features_train, features_test, target_train, target_test = train_test_split(features, target_data, test_size=0.5, shuffle=False)

# データをPyTorchのテンソル形式に変換
features_train_tensor = torch.FloatTensor(features_train.values)
target_train_tensor = torch.FloatTensor(target_train.values)
features_test_tensor = torch.FloatTensor(features_test.values)
target_test_tensor = torch.FloatTensor(target_test.values)

# モデルを定義（入力層、隠れ層、出力層）
hidden_layer_size = 4
neural_network_model = nn.Sequential(
    nn.Linear(features_train_tensor.shape[1], hidden_layer_size),
    nn.ELU(),
    nn.Linear(hidden_layer_size, 1)
)
optimizer = torch.optim.Adam(neural_network_model.parameters(), lr=0.01)

# トレーニングループを設定
num_epochs = 10000

for epoch in range(num_epochs):
    neural_network_model.train()
    optimizer.zero_grad()

    # 順伝播で予測値を計算
    predicted_values = neural_network_model(features_train_tensor)

    # 買い指値を計算（bid = open - pred）
    bid_prices = target_train_tensor[:, 0] - predicted_values.squeeze()

    # シグモイド関数を使って約定確率を計算
    price_difference = bid_prices - target_train_tensor[:, 1]  # bid - low
    execution_probability = torch.sigmoid(price_difference / 50)

    # 期待リターンを計算
    potential_returns = (target_train_tensor[:, 2] - bid_prices) / bid_prices  # (close - bid) / bid
    expected_returns = execution_probability * potential_returns

    # 対数和を計算してケリー損失を求める
    log_returns = torch.log(1 + expected_returns)
    cumulative_log_returns = torch.sum(log_returns)
    kelly_loss = -cumulative_log_returns  # リターンを最大化したいので負の値を使用

    # 逆伝播でモデルを更新
    kelly_loss.backward()
    optimizer.step()

    # 実際のリターンを計算
    executed_mask = price_difference >= 0  # bid価格がlow以上の場合に約定
    actual_returns = torch.zeros_like(potential_returns)
    actual_returns[executed_mask] = potential_returns[executed_mask]
    actual_cumulative = torch.prod(1 + actual_returns)

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {kelly_loss.item():.4f}, Return: {actual_cumulative.item():.4f}', end='\r')

# テストデータでモデルを評価
neural_network_model.eval()
with torch.no_grad():
    predicted_values = neural_network_model(features_test_tensor)
    bid_prices = target_test_tensor[:, 0] - predicted_values.squeeze()  # open - pred

# バックテストを実施
test_ohlcv = ohlcv[-len(bid_prices):]
bid_prices = pd.Series(bid_prices.numpy(), index=test_ohlcv.index, name='bid')
strategy_returns = ((test_ohlcv.cl - bid_prices) / bid_prices).where(test_ohlcv.lo < bid_prices, 0)
strategy_cumulative = (1 + strategy_returns).cumprod()
print(f"Cumulative Return: {strategy_cumulative.iloc[-1]:.4f}")

# ベースライン戦略と比較
fixed_bid_prices = test_ohlcv.op * (1 - 0.005)
fixed_returns = ((test_ohlcv.cl - fixed_bid_prices) / fixed_bid_prices).where(test_ohlcv.lo < fixed_bid_prices, 0)
fixed_cumulative = (1 + fixed_returns).cumprod()

# ATR/2に指値する戦略
atr = talib.ATR(test_ohlcv.hi.values, test_ohlcv.lo.values, test_ohlcv.cl.values)
atr_bid_prices = test_ohlcv.op - atr/2
atr_returns = ((test_ohlcv.cl - atr_bid_prices) / atr_bid_prices).where(test_ohlcv.lo < atr_bid_prices, 0)
atr_cumulative = (1 + atr_returns).cumprod()

hodl_cumulative = test_ohlcv.cl / test_ohlcv.cl.iloc[0]

# 累積リターン倍率をプロット
plt.plot(strategy_cumulative.asfreq('D'), label='ディープラーニング')
plt.plot(atr_cumulative.asfreq('D'), label='ATR/2')
plt.plot(fixed_cumulative.asfreq('D'), label='固定位置（-0.5%）')
plt.plot(hodl_cumulative.asfreq('D'), label='BTC現物ホールド')
plt.title('ディープラーニングを用いた指値位置決定のバックテスト（テスト期間）')
plt.ylabel('累積リターン倍率')
plt.yscale('log')
plt.legend()
plt.show()